In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
MATRIZ_BCH = np.array([[0.2053, 0.7125, 0.4670],
                        [1.8537, -1.2797, -0.4429],
                        [-0.3655, 1.0120, -0.6014]])

print(MATRIZ_BCH)

In [ ]:
def calc_metricas_cohen(X, Y, Z):
    alto, ancho = X.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
    # Aplicar la matriz BCH
    cohen = pixels_xyz @ MATRIZ_BCH.T  # Resultado (N, 3)
    return cohen.reshape(alto, ancho, 3)

def obtener_brillo(X, Y, Z):
    cohen = calc_metricas_cohen(X, Y, Z)
    D, E, F = cohen[:, :, 0], cohen[:, :, 1], cohen[:, :, 2]
    B = np.sqrt(D**2 + E**2 + F**2)
    return B.mean()

In [ ]:
def obtener_brillo_imagen(imagen_rgb):
    R, G, B = cv2.split(imagen_rgb)
    brillo = obtener_brillo(R, G, B)
    return brillo

In [ ]:
def cargar_imagen_rgb(ruta):
    img = cv2.imread(ruta)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def procesar_imagenes_carpeta_bch(
        carpeta_entrada="imagenes",
        carpeta_salida="imagenes_brillo"
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)

        # Añadimos la subcarpeta específica para 'bch'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "bch")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar los brillos
        ruta_txt = os.path.join(carpeta_salida_sub, "info_brillos_bch.txt")

        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Nivel de Brillo (método BCH)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Valores mayores corresponden a mayor brillo.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(".png"):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en rgb
                    img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                    
                    # Calcular brillo
                    brillo = obtener_brillo_imagen(img_rgb)
                    
                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> brillo: {brillo:.4f}\n")

    print("Procesamiento completado con método BCH.")

In [ ]:
procesar_imagenes_carpeta_bch(
    carpeta_entrada="images",
    carpeta_salida="imagenes_procesadas"
)

In [ ]:
# m= valores en el scrollbar entre 0.5 y 1.5 (modificar si vemos que aumenta/disminuye demasiado)

def modificar_brillo(rgb, m0):
    #algoritmo tv_based
    rgb_nuevo = np.array([rgb[0] * m0, rgb[1] * m0, rgb[2] * m0])
    return rgb_nuevo

In [ ]:
def calcular_b_promedio(imagen_rgb, ventana=20):
    # Convertir a espacio de color LAB
    lab = cv2.cvtColor(imagen_rgb, cv2.COLOR_RGB2LAB)

    # Extraer canal L (luminosidad)
    luminosidad = lab[:,:,0].astype(np.float32)

    # Calcular brillo promedio en el vecindario
    kernel = np.ones((ventana, ventana), np.float32) / (ventana * ventana)
    b_promedio = cv2.filter2D(luminosidad, -1, kernel) # -1 para que sea igual a la imagen de entrada

    return b_promedio


def mejora_contraste(imagen_rgb, k, ventana=20):
    """
    Implementa la mejora de contraste preservando las coordenadas cromáticas
    """
    # Convertir a float para cálculos
    imagen_float = imagen_rgb.astype(np.float32) / 255.0
    
    # Convertir a espacio de color LAB
    lab_img = cv2.cvtColor(imagen_float, cv2.COLOR_RGB2LAB)
    
    # Extraer canal L (luminosidad)
    L = lab_img[:,:,0]
    
    # Calcular luminosidad promedio en el vecindario usando la imagen LAB
    b_promedio = calcular_b_promedio(lab_img, ventana)
    
    # Aplicar la fórmula de mejora de contraste
    epsilon = 1e-10
    ratio = np.maximum(L / (b_promedio + epsilon), 0)
    nuevo_L = b_promedio * np.power(ratio, k)
    
    # Limitar valores al rango válido
    nuevo_L = np.clip(nuevo_L, 0, 100)
    
    # Reemplazar canal L
    lab_img[:,:,0] = nuevo_L
    
    # Convertir de vuelta a RGB
    resultado = cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB) * 255.0
    
    # Retornar como uint8
    return np.clip(resultado, 0, 255).astype(np.uint8)

In [ ]:
def procesar_imagenes_contraste(
    carpeta_entrada="images",
    carpeta_salida="imagenes_procesadas",
    valores_k=[0.5, 1.0, 1.5, 2.0]
):

    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'contraste'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "contraste")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
                    
        # Recorremos los archivos dentro de la subcarpeta
        for filename in sorted(os.listdir(ruta_subcarpeta)):
            if filename.lower().endswith("sharp.png"):
                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)

                # Cargar imagen
                img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                if img_rgb is None:
                    continue
                
                nombre, extension = os.path.splitext(filename)
                nombre_salida = f"{nombre}_original.jpg"
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)
                
                # Convertir de RGB a BGR para guardar con OpenCV
                img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
                cv2.imwrite(ruta_imagen_salida, img_bgr)

                # Filtro
                for k in valores_k:
                    contraste_mejorado = mejora_contraste(img_rgb, k)
                    
                    nombre_salida = f"{nombre}_k{k:.1f}.jpg"
                    ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                    # Convertir de RGB a BGR para guardar con OpenCV
                    contraste_mejorado_bgr = cv2.cvtColor(contraste_mejorado, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(ruta_imagen_salida, contraste_mejorado_bgr)

    print("Procesamiento completado con mejora de contraste.")

In [ ]:
# Ejecuta el procesamiento
procesar_imagenes_contraste(
    carpeta_entrada="images",
    carpeta_salida="imagenes_procesadas",
    valores_k=[0.5, 1.0, 1.5, 2.0]
)